<a href="https://colab.research.google.com/github/DivyanshuSheth/dialog-systems-evaluation/blob/main/run_gpt2_lm_cleaned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! rm -r sample_data
! pip -q install transformers
! pip -q install pytorch-lightning
! pip -q install datasets

     |████████████████████████████████| 4.9 MB 9.0 MB/s 
     |████████████████████████████████| 6.6 MB 54.1 MB/s 
     |████████████████████████████████| 120 kB 68.2 MB/s 
     |████████████████████████████████| 708 kB 8.4 MB/s 
     |████████████████████████████████| 419 kB 24.3 MB/s 
     |████████████████████████████████| 5.9 MB 22.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.
     |████████████████████████████████| 431 kB 7.0 MB/s 
     |████████████████████████████████| 115 kB 73.8 MB/s 
     |████████████████████████████████| 212 kB 68.5 MB/s 
     |████████████████████████████████| 127 kB 69.3 MB/s 


In [ ]:
import os
import math
import pandas as pd
import numpy as np
from pprint import pprint
import torch
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
from pytorch_lightning import seed_everything
import pickle as pkl
from scipy.stats import spearmanr, pearsonr
os.chdir("/content/drive/MyDrive/iitkgp-mtp-dialog-response-generation")
RANDOM_SEED = 42
seed_everything(RANDOM_SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


# **Prompts Training**

In [ ]:
prefix = """Human Evaluation of Chatbot Outputs:

Annotation Instructions: 
You will be given a conversation between two individuals. You will then be given a potential chatbot-generated response for the next turn in the conversation. Your task is to rate the response on several metrics. The response for one metric should not influence other metrics. For example, if a response is not understandable or has grammatical errors - you should try to ignore this when considering whether it maintains context or if it is interesting.
The following are the metrics and corresponding rating scales that each response is required to be rated on:
Understandable (0 - 1): Is the response understandable in the context of the history? A score of 0 (no) means that the response is difficult to understand. You do not know what the person is trying to say. A score of 1 (yes) means that the response is understandable. You know what the person is trying to say.
Natural (1 - 3): Is the response naturally written? A score of 1 (bad) means that the response is unnatural. A score of 2 (ok) means the response is strange, but not entirely unnatural. A response of 3 (good) means that the response is natural.
Maintains Context (1 - 3): Does the response serve as a valid continuation of the conversation history? A score of 1 (no) means that the response drastically changes topic or ignores the conversation history. A score of 2 (somewhat) means the response refers to the conversation history in a limited capacity (e.g.,in a generic way) and shifts the conversation topic. A score of 3 (yes) means the response is on topic and strongly acknowledges the conversation history.
Interesting (1 - 3): Is the response dull or interesting? A score of 1 (dull) means that the response is generic and dull. A score of 2 (somewhat interesting) means the response is somewhat interesting and could engage you in the conversation (e.g., an opinion, thought). A score of 3 (interesting) means the response is very interesting or presents an interesting fact.
Uses Knowledge (0 - 1): Given the fact that the response is conditioned on, how well does the response use that fact? A score of 0 (no) means the response does not mention or refer to the fact at all. Ascore of 1 (yes) means the response uses the fact well.
Overall Quality (1 - 5): Given your answers above, what is your overall impression of the generated response/utterance? A score of 1 (very bad) means the response is completely invalid, and it would be difficult to recover the conversation after this. A score of 2 (bad) means that the response is valid, but otherwise poor in quality. A score of 3 (neutral) means the response is neither good nor bad, and has no negative qualities, but no positive ones either. A score of 4 (good) means this is a good response, but falls short of being perfect because of a key flaw. A score of 5 (very good) means this response is good and does not have any strong flaws."""

In [ ]:
with open("/content/drive/MyDrive/iitkgp-mtp-dialog-response-generation/tc_all_prompts.pkl", "rb") as f1:
    tc_all_prompts = pkl.load(f1)

with open("/content/drive/MyDrive/iitkgp-mtp-dialog-response-generation/tc_all_labels.pkl", "rb") as f1:
    tc_all_labels = pkl.load(f1)

with open("/content/drive/MyDrive/iitkgp-mtp-dialog-response-generation/pc_all_prompts.pkl", "rb") as f1:
    pc_all_prompts = pkl.load(f1)

with open("/content/drive/MyDrive/iitkgp-mtp-dialog-response-generation/pc_all_labels.pkl", "rb") as f1:
    pc_all_labels = pkl.load(f1)

pc_all_prompts_labelled = [-1] * len(pc_all_prompts)
tc_all_prompts_labelled = [-1] * len(tc_all_prompts)
for i, each in enumerate(pc_all_prompts):
    prompt_here = pc_all_prompts[i]
    label_1 = round(np.mean(np.array(pc_all_labels[i][0])))
    label_2 = round(np.mean(np.array(pc_all_labels[i][1])))
    label_3 = round(np.mean(np.array(pc_all_labels[i][2])))
    label_4 = round(np.mean(np.array(pc_all_labels[i][3])))
    label_5 = round(np.mean(np.array(pc_all_labels[i][4])))
    label_6 = round(np.mean(np.array(pc_all_labels[i][5])))
    # all_prompts[i] = prefix + each + f"\n1. Understandable: {str(label_1)}\n2. Natural: {str(label_2)}\n3. Maintains Context: {str(label_3)}\n4. Interesting: {str(label_4)}\n5. Overall Quality: {str(label_5)}"
    # pc_all_prompts_labelled[i] = each.lstrip() + f"\n1. Understandable: {str(label_1)}\n2. Natural: {str(label_2)}\n3. Maintains Context: {str(label_3)}\n4. Interesting: {str(label_4)}\n5. Uses Knowledge: {str(label_5)}\n6. Overall Quality: {str(label_6)}"
    pc_all_prompts_labelled[i] = each.lstrip() + f"\n1. {str(label_1)}\n2. {str(label_2)}\n3. {str(label_3)}\n4. {str(label_4)}\n5. {str(label_5)}\n6. {str(label_6)}"

for i, each in enumerate(tc_all_prompts):
    prompt_here = tc_all_prompts[i]
    label_1 = round(np.mean(np.array(tc_all_labels[i][0])))
    label_2 = round(np.mean(np.array(tc_all_labels[i][1])))
    label_3 = round(np.mean(np.array(tc_all_labels[i][2])))
    label_4 = round(np.mean(np.array(tc_all_labels[i][3])))
    label_5 = round(np.mean(np.array(tc_all_labels[i][4])))
    label_6 = round(np.mean(np.array(tc_all_labels[i][5])))
    # all_prompts[i] = prefix + each + f"\n1. Understandable: {str(label_1)}\n2. Natural: {str(label_2)}\n3. Maintains Context: {str(label_3)}\n4. Interesting: {str(label_4)}\n5. Overall Quality: {str(label_5)}"
    # tc_all_prompts_labelled[i] = each.lstrip() + f"\n1. Understandable: {str(label_1)}\n2. Natural: {str(label_2)}\n3. Maintains Context: {str(label_3)}\n4. Interesting: {str(label_4)}\n5. Uses Knowledge: {str(label_5)}\n6. Overall Quality: {str(label_6)}"
    tc_all_prompts_labelled[i] = each.lstrip() + f"\n1. {str(label_1)}\n2. {str(label_2)}\n3. {str(label_3)}\n4. {str(label_4)}\n5. {str(label_5)}\n6. {str(label_6)}"

In [ ]:
print(pc_all_prompts_labelled[0])

Context:
Person 1: lead singer for a band , music teacher
Person 2: wow nice are you really good ?
Person 1: millions of plays on soundcloud
Person 2: really would you share or are you shy

Facts:
Person 1's statement: i also have a dog walking business.
Person 1's statement: i've three dogs.
Person 1's statement: my father was a door to door salesman.
Person 1's statement: i am in an open polyamorous relationship.
Person 1's statement: i like to watch the olympics.

Generated response: 
Person 1: ha ha i'm so shy

Questions about the generated response:
1. Understandable (0 - 1): Is the response understandable given the previous context?
2. Natural (1 - 3): Does the response seem like something that a person would naturally say?
3. Maintains Context (1 - 3): Does the response serve as a valid continuation of the preceding conversation?
4. Interesting (1 - 3): Is the response dull or interesting?
5. Uses Knowledge (0 - 1): Given the facts that the response is conditioned on, how well d

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPTNeoForCausalLM, GPTNeoModel, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
import pandas as pd
from torch.utils.data import Dataset, random_split
import random
from torch import cuda
import os
import numpy as np

In [ ]:
# Set the random seed to a fixed value to get reproducible results 
# torch.manual_seed(42)
# Download the pre-trained GPT-Neo model's tokenizer
# Add the custom tokens denoting the beginning and the end 
# of the sequence and a special token for padding

# model_checkpoint = "gpt2-medium"
# model_checkpoint = "gpt2-large"
model_checkpoint = "bigscience/bloom-560m"
# model_checkpoint = "bigscience/bloom-1b1"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# tokenizer = GPT2Tokenizer.from_pretrained(model_checkpoint,
#                             bos_token="<|startoftext|>",
#                             eos_token="<|endoftext|>",
#                             pad_token="<|pad|>")

Downloading:   0%|          | 0.00/222 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [ ]:
# model = GPT2LMHeadModel.from_pretrained(model_checkpoint).to(device)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)
# model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B").cuda()
# Resize the token embeddings because we've just added 3 new tokens 
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

Downloading:   0%|          | 0.00/710 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
tc_lines = tc_all_prompts_labelled
pc_lines = pc_all_prompts_labelled
max_length = 1024
tc_descriptions = [description for description in tc_lines if len(tokenizer.encode(description)) < max_length-2]
pc_descriptions = [description for description in pc_lines if len(tokenizer.encode(description)) < max_length-2]
pc_max_length = max([len(tokenizer.encode(description)) for description in pc_descriptions])
tc_max_length = max([len(tokenizer.encode(description)) for description in tc_descriptions])
print(pc_max_length)
print(len(pc_descriptions))
print(tc_max_length)
print(len(tc_descriptions))

371
300
823
360


In [ ]:
class PromptsDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            # Encode the descriptions using the GPT-Neo tokenizer
            encodings_dict = tokenizer("<|startoftext|>" 
                                        + txt +    
                                        "<|endoftext|>",
                                        truncation=True,
                                        max_length=max_length, 
                                        padding='max_length')
            input_ids = torch.tensor(encodings_dict['input_ids'])    
            self.input_ids.append(input_ids)
            mask = torch.tensor(encodings_dict['attention_mask'])
            self.attn_masks.append(mask)
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]


tc_dataset = PromptsDataset(tc_descriptions, tokenizer, max_length)
pc_dataset = PromptsDataset(pc_descriptions, tokenizer, max_length)

In [ ]:
pc_train_size = int(0.7 * len(pc_dataset))
tc_train_size = int(0.7 * len(tc_dataset))

pc_train_dataset, pc_trainval_dataset = random_split(pc_dataset, 
                            [pc_train_size, len(pc_dataset) - pc_train_size],
                            generator=torch.Generator().manual_seed(RANDOM_SEED))

pc_test_size = int(0.5 * len(pc_trainval_dataset))
pc_test_dataset, pc_val_dataset = random_split(pc_trainval_dataset, 
                            [pc_test_size, len(pc_trainval_dataset) - pc_test_size],
                            generator=torch.Generator().manual_seed(RANDOM_SEED))

tc_train_dataset, tc_trainval_dataset = random_split(tc_dataset, 
                            [tc_train_size, len(tc_dataset) - tc_train_size],
                            generator=torch.Generator().manual_seed(RANDOM_SEED))
tc_test_size = int(0.5 * len(tc_trainval_dataset))
tc_test_dataset, tc_val_dataset = random_split(tc_trainval_dataset, 
                            [tc_test_size, len(tc_trainval_dataset) - tc_test_size],
                            generator=torch.Generator().manual_seed(RANDOM_SEED))

# test_size = int(0.5 * len(trainval_dataset))
# val_dataset, test_dataset = random_split(trainval_dataset, [test_size, len(trainval_dataset) - test_size])
# train_dataset = dataset

In [ ]:
_, pc_unlabelled_trainval = random_split(pc_all_prompts,
                                         [pc_train_size, len(pc_dataset) - pc_train_size],
                                         generator=torch.Generator().manual_seed(RANDOM_SEED))
pc_unlabelled_test, pc_unlabelled_val = random_split(pc_unlabelled_trainval,
                                                     [pc_test_size, len(pc_trainval_dataset) - pc_test_size],
                                                     generator=torch.Generator().manual_seed(RANDOM_SEED))

_, tc_unlabelled_trainval = random_split(tc_all_prompts, 
                                         [tc_train_size, len(tc_dataset) - tc_train_size],
                                         generator=torch.Generator().manual_seed(RANDOM_SEED))

tc_unlabelled_test, tc_unlabelled_val = random_split(tc_unlabelled_trainval, 
                                                     [tc_test_size, len(tc_trainval_dataset) - tc_test_size],
                                                     generator=torch.Generator().manual_seed(RANDOM_SEED))

In [ ]:
print(tokenizer.decode(tc_train_dataset[0][0], skip_special_tokens=True))

In [ ]:
print(tc_val_labels[4])

In [ ]:
pc_train_labels, pc_trainval_labels = random_split(pc_all_labels, 
                                              [pc_train_size, len(pc_dataset) - pc_train_size],
                                              generator=torch.Generator().manual_seed(RANDOM_SEED))
pc_test_labels, pc_val_labels = random_split(pc_trainval_labels,
                                             [pc_test_size, len(pc_trainval_labels) - pc_test_size],
                                             generator=torch.Generator().manual_seed(RANDOM_SEED))

tc_train_labels, tc_trainval_labels = random_split(tc_all_labels, 
                                              [tc_train_size, len(tc_dataset) - tc_train_size],
                                              generator=torch.Generator().manual_seed(RANDOM_SEED))

tc_test_labels, tc_val_labels = random_split(tc_trainval_labels,
                                             [tc_test_size, len(tc_trainval_labels) - tc_test_size],
                                             generator=torch.Generator().manual_seed(RANDOM_SEED))

In [ ]:
print(tc_val_labels[0])

In [ ]:
print(tokenizer.decode(pc_train_dataset[0][0], skip_special_tokens=True))

In [ ]:
len(tc_train_dataset)

251

In [ ]:
print(len(pc_train_dataset))
print(len(tc_train_dataset))

In [ ]:
print(tokenizer.decode(tc_pc_concatdataset[251][0], skip_special_tokens=True))

In [ ]:
tc_pc_concatdataset = torch.utils.data.ConcatDataset([tc_train_dataset, pc_train_dataset])

In [ ]:
lr=0.5e-5
bs=10
ne=4
training_args = TrainingArguments(output_dir=f'bloom-tc-medium-lm-lr{lr}-bs{bs}-ne{ne}',
                                  overwrite_output_dir=True,
                                  num_train_epochs=ne,
                                  learning_rate=lr,
                                  save_strategy='epoch',
                                  evaluation_strategy='epoch',
                                #   logging_steps=200,
                                #   save_steps =1000,
                                  per_device_train_batch_size=1,
                                  gradient_accumulation_steps=bs,
                                #   per_device_eval_batch_size=2,
                                #   warmup_steps=500,
                                  weight_decay=0.01,  
                                  load_best_model_at_end=True,
                                  fp16=True,
                                  )
                                #   logging_dir='./logs')

In [ ]:
trainer = Trainer(model=model, args=training_args,  
                  train_dataset=tc_pc_concatdataset,
                  eval_dataset=tc_val_dataset, 
                  # This custom collate function is necessary 
                  # to built batches of data
                  data_collator=lambda data: 
              {'input_ids': torch.stack([f[0] for f in data]),       
               'attention_mask': torch.stack([f[1] for f in data]),
               'labels': torch.stack([f[0] for f in data])}
                 )

Using cuda_amp half precision backend


In [ ]:
# Start training process!
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 461
  Num Epochs = 4
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 10
  Total optimization steps = 184


Epoch,Training Loss,Validation Loss


RuntimeError: ignored

In [ ]:
model_checkpoint = f'gpt2-tc-medium-lm-lr{lr}-bs{bs}-ne{ne}/checkpoint-46'
model = GPT2LMHeadModel.from_pretrained(model_checkpoint).to(device)
# model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B").cuda()
# Resize the token embeddings because we've just added 3 new tokens 
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

loading configuration file gpt2-tc-medium-lm-lr1e-05-bs10-ne4/checkpoint-46/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2-medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "pad_token_id": 50258,
  "predict_special_tokens": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "to

In [ ]:
each = """Context:
Person 1:  the guy is a machine! but he needs a better stage name. maybe president banana? 
Person 2:  i do not think the president of zimbabwe would be happy about that 
Person 1:  he could be the artist formerly known as president banana? i don't really care as long as i can listen to him on the radio! i love two things on my radio : indonesian pop music and electromagnetic storms from jupiter. those are my go to listening pleasures! 
Person 2:  yeah that is so cool that if you turn your radio to am, you may capture jupiter's storms 

Facts:
according to canadian law, all radios are required to have at least 40 % of the music played be canadian.

Generated response: 
Person 1: that might be banned in canada. the law says that all stations must have 40 % of the music played be canadian!

Questions about the generated response:
1. Understandable (0 - 1): Is the response understandable given the previous context?
2. Natural (1 - 3): Does the response seem like something that a person would naturally say?
3. Maintains Context (1 - 3): Does the response serve as a valid continuation of the preceding conversation?
4. Interesting (1 - 3): Is the response dull or interesting?
5. Uses Knowledge (0 - 1): Given the facts that the response is conditioned on, how well does the response use the facts?
6. Overall Quality (1 - 5): Given your answers above, what is your overall impression of the quality of the generated response?

Answers:
"""
tokenized = tokenizer(each)
input_ids = torch.tensor([tokenized.input_ids]).to(device)
if len(input_ids[0]) < 1000:
    generation = model.generate(input_ids, do_sample=False, max_new_tokens=100, num_beams=5)
    print(tokenizer.batch_decode(generation, skip_special_tokens=True)[0])

Context:
Person 1:  the guy is a machine! but he needs a better stage name. maybe president banana? 
Person 2:  i do not think the president of zimbabwe would be happy about that 
Person 1:  he could be the artist formerly known as president banana? i don't really care as long as i can listen to him on the radio! i love two things on my radio : indonesian pop music and electromagnetic storms from jupiter. those are my go to listening pleasures! 
Person 2:  yeah that is so cool that if you turn your radio to am, you may capture jupiter's storms 

Facts:
according to canadian law, all radios are required to have at least 40 % of the music played be canadian.

Generated response: 
Person 1: that might be banned in canada. the law says that all stations must have 40 % of the music played be canadian!

Questions about the generated response:
1. Understandable (0 - 1): Is the response understandable given the previous context?
2. Natural (1 - 3): Does the response seem like something that a 

In [ ]:
tokenizer.decode(tc_train_dataset[0][0], skip_special_tokens=True)[-200:]

'does the response use the facts?\n6. Overall Quality (1 - 5): Given your answers above, what is your overall impression of the quality of the generated response?\n\nAnswers:\n1. 1\n2. 2\n3. 2\n4. 3\n5. 1\n6. 4'

In [ ]:
tc_unlabelled_test[0][-200:]

'e is conditioned on, how well does the response use the facts?\n6. Overall Quality (1 - 5): Given your answers above, what is your overall impression of the quality of the generated response?\n\nAnswers:'

In [ ]:
tc_test_dataset = tc_unlabelled_test
for i, each in enumerate(tc_test_dataset):
    # print(each)
    tokenized = tokenizer(each + "\n")
    input_ids = torch.tensor([tokenized.input_ids]).to(device)
    input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    if len(input_ids[0]) < 1000:
        generation = model.generate(input_ids, do_sample=False, max_new_tokens=100, num_beams=3, temperature=0.1)
        print(f"\nnum = {i}")
        print(tokenizer.batch_decode(generation, skip_special_tokens=True)[0].replace(input_text, ""))

In [ ]:
str1 = "hey there"
tokenized = tokenizer(str1)
input_ids = torch.tensor([tokenized.input_ids]).to(device)
generation = model.generate(input_ids, do_sample=False, max_new_tokens=100, num_beams=5)
print(tokenizer.batch_decode(generation, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


hey theres


In [ ]:
input_ids = torch.tensor([test_dataset[0][0].tolist()]).to(device)
if len(input_ids[0]) < 1000:
    generation = model.generate(input_ids, do_sample=False, max_new_tokens=24, num_beams=5)
    print(tokenizer.batch_decode(generation, skip_special_tokens=True)[0])

In [ ]:
print(tokenizer.decode(test_dataset[0][0]))

<|startoftext|> Human Evaluation of Chatbot Outputs:

Annotation Instructions: 
You will be given a conversation between two individuals. You will then be given a potential chatbot-generated response for the next turn in the conversation. Your task is to rate the response on several metrics. The response for one metric should not influence other metrics. For example, if a response is not understandable or has grammatical errors - you should try to ignore this when considering whether it maintains context or if it is interesting.
The following are the metrics and corresponding rating scales that each response is required to be rated on:
Understandable (0 - 1): Is the response understandable in the context of the history? A score of 0 (no) means that the response is difficult to understand. You do not know what the person is trying to say. A score of 1 (yes) means that the response is understandable. You know what the person is trying to say.
Natural (1 - 3): Is the response naturally wr

In [ ]:
for each in test_dataset:
    input_ids = torch.tensor([each[0].tolist()]).to('cuda')
    if len(input_ids[0]) < 1000:
        outputs = model.generate(input_ids, do_sample=False, max_new_tokens=24, num_beams=5)
        print(str(tokenizer.decode(input_ids[0])))
        model_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        print(model_response)
        print("\n")
    else:
        print(len(input_ids[0]))

In [ ]:
tokenized['input_ids']

In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id
# prompt = "hey there"
# tokenized = tokenizer(prompt, return_tensors="pt")
# input_ids = tokenized['input_ids'].to('cuda')

outputs = model.generate(torch.tensor([test_dataset[0][0].tolist()]).to('cuda'), do_sample=False, max_new_tokens=100, num_beams=5)
print("Prompt: " + str(tokenizer.decode(input_ids[0])))
model_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
# feedback_starts = model_response.find("Feedback: ")
print("Model Response: " + model_response)
# print()